In [99]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams ["figure.figsize"] = 10,6
import seaborn as sns
from scipy import stats
import math
import datetime
import urllib
import zipfile
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import glob
ohe = OneHotEncoder
le = LabelEncoder()
sc_X = StandardScaler()
from sklearn.compose import ColumnTransformer
le = LabelEncoder()

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression as LR

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import cohen_kappa_score

In [59]:
bike_station = pd.read_csv('stations.csv')


In [68]:
bike_station.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,1


In [61]:
bike_station.loc[bike_station['Member type'] == "Casual", 'Member cat'] = "1"
bike_station.loc[bike_station['Member type'] == "Member", 'Member cat'] = "2"
bike_station.loc[bike_station['Member type'] == "Unknown", 'Member cat'] = "1"

In [62]:
bike_station

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,1
...,...,...,...,...,...,...,...,...,...,...
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,15th & P St NW,31214,17th & Corcoran St NW,W01459,Member,2
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,Convention Center / 7th & M St NW,31201,15th & P St NW,W01262,Member,2
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,Lamont & Mt Pleasant NW,31602,Park Rd & Holmead Pl NW,W00998,Member,2
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,21st & I St NW,31222,New York Ave & 15th St NW,W00042,Member,2


In [70]:
bike_station["Member cat"].info


<bound method Series.info of 0          2
1          1
2          2
3          2
4          1
          ..
1226762    2
1226763    2
1226764    2
1226765    2
1226766    2
Name: Member cat, Length: 1226767, dtype: object>

In [65]:
#fix the hot encoding

print(ohe)

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [97]:
new_station = pd.DataFrame(bike_station, columns=['Member cat'])

new_station['Member cat'] = new_station['Member cat'].astype('category')

new_station['Member cat new'] = new_station['Member cat'].cat.codes

new_station

,Member cat,Member cat new
0,2,1
1,1,0
2,2,1
3,2,1
4,1,0
...,...,...
1226762,2,1
1226763,2,1
1226764,2,1
1226765,2,1


In [103]:
x_data = bike_station[["Duration", "Start station number", "End station number"]] #change to something more useful like start date and end date. Fix the dates though, first. 
y_data = new_station["Member cat new"]

#need to change member type. C=1,  M=2. Unknown are also 1.

In [102]:
test_frame = [bike_station, new_station]
test_frame

[         Duration           Start date             End date  \
 0            3548  2011-01-01 00:01:29  2011-01-01 01:00:37   
 1             346  2011-01-01 00:02:46  2011-01-01 00:08:32   
 2             562  2011-01-01 00:06:13  2011-01-01 00:15:36   
 3             434  2011-01-01 00:09:21  2011-01-01 00:16:36   
 4             233  2011-01-01 00:28:26  2011-01-01 00:32:19   
 ...           ...                  ...                  ...   
 1226762       300  2011-12-31 23:41:19  2011-12-31 23:46:20   
 1226763       387  2011-12-31 23:46:43  2011-12-31 23:53:10   
 1226764       261  2011-12-31 23:47:27  2011-12-31 23:51:49   
 1226765      2060  2011-12-31 23:55:12  2012-01-01 00:29:33   
 1226766       468  2011-12-31 23:55:56  2012-01-01 00:03:45   
 
          Start station number                      Start station  \
 0                       31620                      5th & F St NW   
 1                       31105               14th & Harvard St NW   
 2                     

In [104]:

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.3, shuffle=False)
x_train, x_test, y_train, y_test

(        Duration  Start station number  End station number
 0           3548                 31620               31620
 1            346                 31105               31101
 2            562                 31400               31104
 3            434                 31111               31503
 4            233                 31104               31106
 ...          ...                   ...                 ...
 858731       785                 31236               31225
 858732       495                 31227               31604
 858733       701                 31219               31109
 858734       180                 31111               31101
 858735      1079                 31219               31213
 
 [858736 rows x 3 columns],
          Duration  Start station number  End station number
 858736       1204                 31503               31237
 858737       1202                 31503               31237
 858738       2580                 31215               31215
 85873

In [105]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data,test_size=0.2, random_state=42)

In [106]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing data
X_test_norm = norm.transform(X_test)

In [107]:
bike_station.isnull().any()

#I need to drop the Unknown ones...done.

Duration                False
Start date              False
End date                False
Start station number    False
Start station           False
End station number      False
End station             False
Bike number             False
Member type             False
Member cat              False
dtype: bool

In [108]:
reg = LR().fit(x_data, y_data)

In [109]:
reg.score(x_data, y_data)

0.10754912433886676

In [110]:
y_predicted = reg.predict(x_test)

In [111]:
reg.coef_

array([-5.25730215e-05,  4.36012701e-05,  5.50321730e-05])

In [112]:
reg.intercept_

-2.2253248156715673

In [113]:
reg.predict #(np.array([[3, 5]])) ex.

<bound method LinearModel.predict of LinearRegression()>

In [114]:
y_predicted = reg.predict(x_test)
y_predicted

array([0.80398807, 0.80409321, 0.71787972, ..., 0.85638507, 0.7451669 ,
       0.8234522 ])